# Integrando Python com SQLAlchemy

Desafio de Projeto - Implementar uma aplicação de integração com SQLite com base em um esquema relacional disponibilizado. Sendo assim, utilize o esquema dentro do contexto de cliente e conta para criar as classes de sua API. Essas classes irão representar as tabelas do banco de dados relacional dentro da aplicação.

- Aplicação com a definição do esquema por meio das classes usando SQLAlchemy
- Inserção de um conjunto de dados mínimo para manipulação das informações
- Execução de métodos de recuperação de dados via SQLAlchemy

### Implementando um Banco de Dados Relacional com SQLAlchemy

In [ ]:
!pip install sqlalchemy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sqlalchemy.sql.lambdas import insp
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.orm import Session
from sqlalchemy import Column
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy import ForeignKey
from sqlalchemy.types import CHAR
from sqlalchemy import Float
from sqlalchemy import select
from sqlalchemy import func

In [ ]:
Base = declarative_base()


class Cliente(Base):
  __tablename__ = "cliente"
  id = Column(Integer, primary_key=True)
  nome = Column(String)
  cpf = Column(String)
  endereco = Column(String)

  conta = relationship(
      "Conta", back_populates="cliente", cascade="all, delete-orphan"
  )

  def __repr__(self):
    return f"Cliente(id={self.id}, nome={self.nome}, cpf={self.cpf}, endereco={self.endereco})"

class Conta(Base):
  __tablename__ = "conta"
  id = Column(Integer, primary_key=True)
  tipo = Column(String(10), nullable=False)
  numero = Column(String(10), nullable=False)
  agencia = Column(Integer, nullable=False)
  saldo = Column(Float, nullable=False)
  cliente_id = Column(Integer, ForeignKey("cliente.id"), nullable=False)

  cliente = relationship("Cliente", back_populates="conta")
  
  def __repr__(self):
    return f"Conta(id={self.id}, tipo={self.tipo}, numero={self.numero}, agencia={self.agencia}, saldo={self.saldo})"

In [ ]:
print(Cliente.__tablename__)
print(Conta.__tablename__)

cliente
conta


- Conexão 

In [ ]:
engine = create_engine("sqlite://")

- Criando as classes como tabelas no banco de dados

In [ ]:
Base.metadata.create_all(engine)

- Printando o esquema do banco de dados

In [ ]:
engine_inspector = inspect(engine)

print(engine_inspector.has_table("cliente"))
print(engine_inspector.get_table_names())
print(engine_inspector.default_schema_name)

True
['cliente', 'conta']
main


### Persistência dos dados

In [ ]:
with Session(engine) as session:
  cliente_1 = Cliente(
      nome='Juliana',
      cpf=12345678911,
      endereco='Rua 1, 10 - Rio de Janeiro/RJ',
      conta=[Conta(tipo='Poupança', agencia='0001', numero=20055001, saldo=10000)]
  )

  cliente_2 = Cliente(
      nome='Venilton',
      cpf=14725836901,
      endereco='Rua 2, 20 - Araraquara/SP',
      conta=[Conta(tipo='Corrente', agencia='0001', numero=10055001, saldo=7800)]
  )
  
  cliente_3 = Cliente(
      nome='Renan',
      cpf=74185296302,
      endereco='Rua 3, 30 - Araraquara/SP',
      conta=[Conta(tipo='Corrente', agencia='0001', numero=10055003, saldo=8000)]
  )
  
  cliente_4 = Cliente(
      nome='Guilherme',
      cpf=31264597809,
      endereco='Rua 4, 40 - Araraquara/SP',
      conta=[Conta(tipo='Poupança', agencia='0001', numero=20055004, saldo=1200)]
  )
  
  cliente_5 = Cliente(
      nome='Diego',
      cpf=21354687910,
      endereco='Rua 5, 50 - Catanduva/SP',
      conta=[Conta(tipo='Corrente', agencia='0001', numero=10055007, saldo=1200)]
  )

- Persistindo dados para banco de dados

In [ ]:
session.add_all([cliente_1, cliente_2, cliente_3, cliente_4, cliente_5])

In [ ]:
session.commit()

 ## Consumindo informações do BD

- Recuperando usuários a partir de condição de filtragem

In [ ]:
stmt = select(Cliente).where(Cliente.nome.in_(['Guilherme', 'Juliana', 'Diego']))
for cliente in session.scalars(stmt):
  print(cliente)

Cliente(id=1, nome=Juliana, cpf=12345678911, endereco=Rua 1, 10 - Rio de Janeiro/RJ)
Cliente(id=4, nome=Guilherme, cpf=31264597809, endereco=Rua 4, 40 - Araraquara/SP)
Cliente(id=5, nome=Diego, cpf=21354687910, endereco=Rua 5, 50 - Catanduva/SP)


- Recuperando os endereços de Renan

In [ ]:
conta_stmt = select(Conta).where(Conta.cliente_id.in_([3]))
for conta in session.scalars(conta_stmt):
  print(conta)

Conta(id=3, tipo=Corrente, numero=10055003, agencia=1, saldo=8000.0)


- Recuperando informações de maneira ordenada

In [ ]:
print(select(Cliente).order_by(Cliente.nome.desc()))

SELECT cliente.id, cliente.nome, cliente.cpf, cliente.endereco 
FROM cliente ORDER BY cliente.nome DESC


In [ ]:
order_stmt = select(Cliente).order_by(Cliente.endereco.desc())
for i, result in enumerate (session.scalars(order_stmt)):
  print(f'Resultado {i}: {result}')

Resultado 0: Cliente(id=5, nome=Diego, cpf=21354687910, endereco=Rua 5, 50 - Catanduva/SP)
Resultado 1: Cliente(id=4, nome=Guilherme, cpf=31264597809, endereco=Rua 4, 40 - Araraquara/SP)
Resultado 2: Cliente(id=3, nome=Renan, cpf=74185296302, endereco=Rua 3, 30 - Araraquara/SP)
Resultado 3: Cliente(id=2, nome=Venilton, cpf=14725836901, endereco=Rua 2, 20 - Araraquara/SP)
Resultado 4: Cliente(id=1, nome=Juliana, cpf=12345678911, endereco=Rua 1, 10 - Rio de Janeiro/RJ)


### Join statment

In [ ]:
print(select(Cliente.nome, Conta.saldo).join_from(Conta, Cliente))

SELECT cliente.nome, conta.saldo 
FROM conta JOIN cliente ON cliente.id = conta.cliente_id


In [ ]:
join_stmt = select(Cliente.cpf, Conta.numero).join_from(Conta, Cliente)

for result in session.scalars(join_stmt):
  print(result)

12345678911
14725836901
74185296302
31264597809
21354687910


- Utilizando o método fetch all

In [ ]:
connection = engine.connect()
results = connection.execute(join_stmt).fetchall()
print('Executando statement a partir da connection. CPFs e Contas\n')
for result in results:
  print(result)

Executando statement a partir da connection. CPFs e Contas

('12345678911', '20055001')
('14725836901', '10055001')
('74185296302', '10055003')
('31264597809', '20055004')
('21354687910', '10055007')


In [ ]:
print(select(func.count('*')).select_from(Cliente))

SELECT count(:count_2) AS count_1 
FROM cliente


- Total de instâncias em Clientes

In [ ]:
count_stmt = select(func.count('*')).select_from(Cliente)
for result in session.scalars(count_stmt):
  print(f'Tota de instâncias em Clientes: {result}')

Tota de instâncias em Clientes: 5


In [ ]:
join_stmt = select(Conta.saldo).join_from(Cliente, Conta).where(Cliente.nome=='Juliana')

for result in session.scalars(join_stmt):
  print(f"R${result:,.2f}")

R$10,000.00
